Plan is to create ASCII like art, using the benefits that a typewriter offers, but being aware of it's limitations. Namely, we can overlay characters to multiply the # of glyphs But, we have limited options to begin with, ie. no |@#

Process:
- Input image, desired output size, and directory of glyph images
    - Sub-division of letter spaces, as well as # glyphs per space
- Image is converted to greyscale, and scaled to output size
- Preprocess glyphs, making greyscale & scaled to sub-division
    - I THINK that scaling to eg 3x3 would be the same as doing manual averages
        - !Want to check this!
        - Current theory is this is what BOX resampling would achieve for us
    - If >1 glyph per space, calculate composite glyphs (This scales scarily)
- From the image, take blocks of pixels, the 'target'
    - Iterate through all the glyphs, working out 'distance' from target
        - Don't forget the black space 'glyph'
        - Distance metric likely to be N dimensional pythagorean distance (RMS)
    - Closest glyph gets chosen, repeat for all targets
- Render preview of design, using the original glyph images
    - Monospace REALLY helps with this (25 x 48)
- Produce instruction set to replicate on the typewriter
    - Developing a nice notation here will be useful to do

CURRENT PROBLEMS

- Currently we're scaling image to full range of our glyphs.
    - This fixes the issues related to have thin glyphs on small SAMPLE_X values
    - However, for higher SAMPLE_X, we can end up making the image darker?
    - Contrast vs Brightness (Perhaps a ranking system rather than Euc distance?)
    
- Image may look MUCH better, if shifted by 1 sample width
    - May want to iterate over, shifting the image, and summing the distances
    - This'd give us a metric of 'best match'
    - Test case would be an image made from glyphs, but shifted a little
        - off anything other than exactly a # of sample widths wouldn't help though
        - Can't check for each and every pixel offset

- Old fill_range was broken, fixed that
    - new fill range produces even more junky images
    - Before, was clipping a lot of lighter colors to 255
    - Now only lightest goes to 255, makes everything bleh
    
    - old was `lambda val: ((val-min_) * (t_max/max_))+ t_min`
    - new is `lambda val: ((val-min_) * (t_range/range_))+t_min`

In [ ]:
import os
from PIL import Image, ImageChops, ImageOps
import numpy as np
import time
from scipy.spatial import cKDTree
import functools
import operator
import itertools
import json
from contextlib import suppress

GLYPH_DIR = 'E:/Users/Richard/Documents/One off mini projects/Typewriting/Typearter/Glyphs'
SAMPLE_X = 3
SAMPLE_Y = 3
TARGET_WIDTH = 60
TARGET_HEIGHT = 60  # This and width will come from image dimensions, or will affect that through cropping

class glyph:
    def __init__(self, name=None, image=None, components=None):
        self.name = name
        self.image = image
        self.fingerprint = self.image.convert("L")\
        .resize((SAMPLE_X, SAMPLE_Y), Image.BOX)
        self.fingerdisplay = self.fingerprint.resize(self.image.size)
        
        if components:
            self.components = components
        else:
            self.components = [self]
        
    @classmethod
    def from_file(cls, filename):
        name = os.path.splitext(filename)[0]
        # looks for name map, and any name alias
        # TODO: probably better in whatever will be making the glyphs
        # --> perhaps an extra override_name arguement that defaults to None
        with suppress(FileNotFoundError):
            with open(os.path.join(GLYPH_DIR, 'name_map.json'), 'r') as fp:
                glyph_names = json.load(fp)
                name = glyph_names.get(name, name)
        image = Image.open(os.path.join(GLYPH_DIR, filename))
        return cls(name=name, image=image)
    
    def __add__(self, other):
        if not isinstance(other, glyph):
            raise TypeError('can only combine glyph (not "{}") with glyph'.format(type(other)))
        name = self.name + ' ' + other.name
        composite = ImageChops.darker(self.image, other.image)
        # keep track of component glyphs, sorted on names
        # debatable if repeats should be kept
        components = sorted(self.components + other.components, key=lambda g: g.name)
        return glyph(name=name, image=composite, components=components) 
    
    def __str__(self):
        return self.name

def chunk(list_, width, chunk_width, chunk_height):
    # Given an sequence, width, a chunk width, and chunk height
    # Will return a list of 'chunks' of length chunk width * chunk height
    # [0,1,2,3,4,5,6,7,8,9,A,B,C,D,E,F], 4, 2, 2
    # 0 1 2 3
    # 4 5 6 7
    # 8 9 A B
    # C D E F
    # Would chunk to:
    # [[0,1,4,5], [2,3,6,7], [8,9,C,D], [A,B,E,F]]
    chunks = []
    height = len(list_) // (width * chunk_height * chunk_width)
    for y in range(height):
        rows = range(chunk_height * y, chunk_height * (y + 1))
        for x in range(width):
            columns = range(chunk_width * x, chunk_width*(x + 1))
            chunk = [list_[column + row * width * chunk_width]\
                     for row in rows for column in columns]
            chunks.append(chunk)
    return chunks

def find_closest_glyph(target):
    dd, ii = tree.query(target)
    return glyph_list[ii]

def load_glyphs(directory):
    glyphs = {}
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            glyph_ = glyph.from_file(filename)
            glyphs.update({glyph_.name: glyph_})
    return glyphs
  
def combine_glyphs(glyphs, depth):
    # returns all possible combinations of 'depth' glyphs
    glyph_combinations = itertools.combinations(iter(glyphs.values()), 2)
    output = {}
    for combination in glyph_combinations:
        output.update(functools.reduce(operator.add, combination))
    return output
    
def average_glyph_value(glyphs):
    # Takes a dict of glyphs, and returns a list of average values
    average_values = []
    for name, glyph in glyphs.items():
        vals = list(glyph.fingerprint.getdata())
        average_value = sum(vals) / len(vals)  # May want to change to int division
        average_values.append(average_value)
    return average_values

def lightest_color(glyphs):
    lightest = 0
    for name, glyph in glyphs.items():
        dark, light = glyph.fingerprint.getextrema()
        lightest = max(light, lightest)
    return lightest

def instructions(result_glyphs):
    instructions = []
    # From the results, produce a set of instructions
    # Instruction strings are 1 per line of image
    # TODO: Support for overstruck characters
    # ---> New line, denoted with an 'a' etc. use space to pad the single chars   
    lines = [result_glyphs[i * TARGET_WIDTH: (i + 1) * TARGET_WIDTH] for i in range(TARGET_HEIGHT)]
    for line in lines:
        glyph_groups = itertools.groupby(line, key=lambda glyph:glyph.name)
        groups = [str(len(list(group))) + key for key, group in glyph_groups]
        instructions.append(' '.join(groups))
    print('\n'.join(instructions))
    
def compose_calculation(result):
    calculation = Image.new("L", (TARGET_WIDTH * 25, TARGET_HEIGHT * 48))
    for i, glyph_ in enumerate(result):
        w = 25
        h = 48
        x = w * (i % TARGET_WIDTH)
        y = h * (i // TARGET_WIDTH)
        calculation.paste(glyph_.fingerdisplay, (x, y, x + w, y + h))
    return calculation
    
def compose_output(result):
    output = Image.new("L", (TARGET_WIDTH * 25, TARGET_HEIGHT  * 48))
    for i, glyph_ in enumerate(result):
        w = 25
        h = 48
        x = w * (i % TARGET_WIDTH)
        y = h * (i // TARGET_WIDTH)
        output.paste(glyph_.image, (x, y, x + w, y + h))
    return output

def fit_to_aspect(image, aspect_ratio):
    # Crops the image around the center to fit aspect ratio
    current_aspect = image.width/image.height
    if current_aspect < desired_aspect:  # Image too tall
        perfect_height = image.width / desired_aspect
        edge = (image.height - perfect_height) /2
        image = image.crop((0, edge, image.width, perfect_height+edge))
    else:  # Image too wide
        perfect_width = image.height * desired_aspect
        edge = (image.width - perfect_width) /2
        image = image.crop((edge, 0, perfect_width+edge, image.height))
    
    return image

def fill_range(image, range_):
    # Scales image colors to fill range_
    min_, max_ = image.getextrema()
    t_min, t_max = range_
    range_ = max_ - min_
    t_range = t_max - t_min
    image =  image.point(lambda val: ((val-min_) * (t_range / range_)) + t_min)
    return image

def equalize_glyph(image, mask=None):
    # Manipulates image histogram to closely resemble that of glyphs
    h = image.histogram(mask)
    target_indices = []
    for i in range(256):
        count = average_vals.count(i)
        if count:
            target_indices.extend([i] * count)

    histo = [_f for _f in h if _f]
    step = (functools.reduce(operator.add, histo) - histo[-1]) // len(target_indices)
            
    lut = []
    n = step//2
    for i in range(256):
        position = min(n//step, len(target_indices)-1)
        lut.append(target_indices[position])
        n += h[i]
    
    return image.point(lut)
        
start = time.time()

glyphs = load_glyphs(GLYPH_DIR)
# glyphs.update(combine_glyphs(glyphs, 2))
  
# blank = Image.new("L", (25,48), 'white')
# space = glyph(name='sp', image=blank)
# glyphs.update({'sp':space})
# lightest_value = 255  # need to unhardcode

# Creating tree will occur when glyphs are loaded
# We could defer this 'til use, and just make sure list of glyphs isn't changed?
glyph_list = list(glyphs.values())
glyph_data = [list(glyph.fingerprint.getdata()) for glyph in glyph_list]
tree = cKDTree(glyph_data)

average_vals = average_glyph_value(glyphs)
lightest_value = max(average_vals)
darkest_value = min(average_vals)

TARGET_IMAGE = 'E:/Users/Richard/Documents/One off mini projects/Typewriting/Typearter/dog.png'
dog = Image.open(TARGET_IMAGE)

# final code will have this tucked away from user
desired_aspect = (25 * TARGET_WIDTH) / (48 * TARGET_WIDTH)
dog = fit_to_aspect(dog, desired_aspect)
dog.show()

# Changing the resampling mode here changes how the image ends up
smol_dog = dog.resize((TARGET_WIDTH * SAMPLE_X, TARGET_HEIGHT * SAMPLE_Y), Image.LANCZOS)
smol_dog = smol_dog.convert("L")

# Some ImageOps may help with color reproduction
# Image manipulation is a very finnickey process, may need to have user controls
#smol_dog = ImageOps.autocontrast(smol_dog, cutoff=0)
#smol_dog = ImageOps.equalize(smol_dog)
#smol_dog = fill_range(smol_dog, (darkest_value, lightest_value))
#smol_dog = equalize_glyph(smol_dog)
smol_dog = fill_range(smol_dog, (150, 245))

target_parts = chunk(list(smol_dog.getdata()), width=TARGET_WIDTH,
                     chunk_width=SAMPLE_X, chunk_height=SAMPLE_Y)

result = []

for section in target_parts:   
    result.append(find_closest_glyph(section))
    
calculation = compose_calculation(result)
output = compose_output(result)

calculation.show()
output.show()

end = time.time()
print(end-start)

instructions(result)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

average_vals = []
sub_values = []
for i, glyph_ in glyphs.items():
    vals = list(glyph_.fingerprint.getdata())
    average_val = sum(vals)//len(vals)
#     print(glyph_.name + ': '+str(average_val))
    average_vals.append(average_val)
    sub_values.extend(vals)
    
section_vals = []
    
for section in target_parts:
    average = sum(section)//len(section)
    section_vals.append(average)
    
fig1 = plt.figure(figsize=(18,16))
l, = plt.plot(np.linspace(0, 1, len(average_vals)), sorted(average_vals), 'rx-')
l, = plt.plot(np.linspace(0, 1, len(section_vals)), sorted(section_vals), 'bx-')
l, = plt.plot(np.linspace(0, 1, len(sub_values)), sorted(sub_values), 'gx-')

In [ ]:
# PLAYING WITH ENHANCEMENTS, TRYING TO MATCH OLD 'LOOK'

from PIL import ImageEnhance, ImageOps
import matplotlib.pyplot as plt

testdog = dog.convert("L").resize((300,600))

#testdog.show()
t_min = darkest_value
t_max = lightest_value
min_, max_ = testdog.getextrema()
range_ = max_ - min_
t_range = t_max - t_min
old_dog = testdog.point(lambda val: ((val-min_) * (t_max/max_))+ t_min)
new_dog =  testdog.point(lambda val: ((val-min_)* (t_range/range_))+t_min)

equal_dog = ImageOps.equalize(testdog)

contrast = ImageEnhance.Contrast(new_dog)
e_dog = contrast.enhance(2.5)
brightness = ImageEnhance.Brightness(e_dog)
b_dog = brightness.enhance(1.3)

equal_scale = fill_range(equal_dog, (t_min, t_max))

%matplotlib inline
f, axarr = plt.subplots(2, 6, figsize=(16,8))
axarr[0][0].imshow(testdog.convert("RGB"))
axarr[0][1].imshow(old_dog.convert("RGB"))
axarr[0][2].imshow(new_dog.convert("RGB"))
axarr[0][3].imshow(b_dog.convert("RGB"))
axarr[0][4].imshow(equal_dog.convert("RGB"))
axarr[0][5].imshow(equal_scale.convert("RGB"))

axarr[1][0].hist(list(testdog.getdata()))
axarr[1][1].hist(list(old_dog.getdata()))
axarr[1][2].hist(list(new_dog.getdata()))
axarr[1][3].hist(list(b_dog.getdata()))
axarr[1][4].hist(list(equal_dog.getdata()))
axarr[1][5].hist(list(equal_scale.getdata()))

subhist = []
for i in range(255):
    subhist.append(sub_values.count(i))

In [ ]:
import json
import os

chars = {
 'and':'&',
 'apos':"'",
 'cdot':'·',
 'colon':':',
 'comma':',',
 'dollar':'$',
 'dot':'.',
 'equal':'=',
 'hyphen':'-',
 'lbrac':'(',
 'percent':'%',
 'plus':'+',
 'pound':'£',
 'question':'?',
 'quote':'"',
 'rbrac':')',
 'semicolon':';',
 'slash':'/',
 'under':'_'
}

for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    chars.update({letter + 'u':letter})
    
print(chars)

path = os.path.join(GLYPH_DIR, 'name_map.json')

with open(path,'w') as fp:
    json.dump(chars, fp, ensure_ascii=False, sort_keys=True, indent=4)